---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

## 🚀 Ejercicio 3: Extensiones Avanzadas (AVANZADO)
1. Implementa procesamiento en lote de múltiples textos
2. Crea una función de exportación de resultados a CSV
3. Desarrolla un sistema de filtrado por tipo de entidad

## 💡 Proyecto Integrador
Elegí una de estas aplicaciones y desarróllala:
- **Analizador de noticias argentinas**: Extrae personas y lugares de artículos
- **Procesador de CVs**: Identifica nombres, empresas y universidades
- **Análisis de redes sociales**: Detecta menciones de políticos y lugares

## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
2. ¿En qué casos usarías un modelo local vs una API?
3. ¿Cómo evaluarías la precisión de los resultados?
4. ¿Qué consideraciones éticas debemos tener en cuenta?
5. ¿Cómo escalarías esta solución para procesar miles de documentos?

## Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [ ]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Todas las dependencias instaladas correctamente


## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

API Key cargada desde Google Colab Secrets


In [ ]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

Cliente Gemini configurado correctamente


In [ ]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [ ]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

🖥️  Dispositivo: CPU
📥 Cargando modelo de NER para español...


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Modelo mrm8488/bert-spanish-cased-finetuned-ner cargado exitosamente


# **🎓 EJERCICIOS**



---


## --- Ejercicio 1: Personalización (BÁSICO) ---


---



In [ ]:
# 1. Modificar los ejemplos para incluir más contexto argentino específico
texto_ejemplo_1 = """
Hola, soy Roberto González y vivo en el barrio de Palermo, Buenos Aires.
Trabajo en la Facultad de Derecho de la Universidad de Buenos Aires.
El sábado pasado fui a un show de tango en San Telmo con mi amiga Mariana.
"""

texto_ejemplo_2 = """
La ciudad de Rosario es muy conocida por su Monumento a la Bandera.
Mi primo Diego vive cerca del río Paraná, en el barrio de Pichincha.
Recientemente, visitamos el parque de España, uno de los lugares más hermosos de la ciudad.
"""

# 2. Agregar 3 ejemplos nuevos con nombres de barrios porteños
texto_ejemplo_3 = """
Soy Ana Martínez y vivo en el barrio de Caballito, en Buenos Aires.
Trabajo en el Hospital de Clínicas, cerca de la estación de subte.
El domingo fui a tomar un café con mis amigos en el barrio de Recoleta.
"""

texto_ejemplo_4 = """
Carlos Fernández trabaja en el barrio de La Boca, en el puerto de Buenos Aires.
El domingo fuimos a caminar por la costanera, una de las zonas más turísticas.
También visitamos el famoso estadio de Boca Juniors.
"""

texto_ejemplo_5 = """
La Casa Rosada está ubicada en el barrio de Monserrat, en pleno centro de Buenos Aires.
Recientemente, un turista italiano me preguntó por el Obelisco, que está muy cerca de la Plaza de la República.
"""

# 3. Probar con texto de diferentes regiones de Argentina
texto_regional_1 = """
En Mendoza, la cosecha de uvas es uno de los eventos más importantes del año.
Mi amigo Pablo tiene una bodega en el Valle de Uco, donde cultivan algunas de las mejores uvas Malbec.
"""

texto_regional_2 = """
En la Patagonia, el clima es muy frío, pero el paisaje es impresionante.
Fuimos a caminar por el Parque Nacional Los Glaciares, cerca de El Calafate, donde vimos el famoso glaciar Perito Moreno.
"""




---


## -- Ejercicio 2: Análisis Comparativo (INTERMEDIO) --


---



In [ ]:
#  Importamos librerías necesarias
import time  # Para medir el tiempo de ejecución de cada modelo

# 1. Función que cuenta cuántas entidades encuentra un modelo
def contar_entidades_por_tipo(texto, modelo, tipo_modelo="transformers"):
    """
    Cuenta entidades por categoría, según el tipo de modelo NER.
    - 'transformers': modelos de Hugging Face
    - 'gemini': función que devuelve string formateado por línea
    """
    if not modelo or not texto.strip():
        return {}  # Si no hay modelo o texto, devuelve diccionario vacío

    entidades = modelo(texto)  # Aplica el modelo al texto
    conteo = {}  # Diccionario para acumular cantidades por tipo de entidad

    #  Caso: modelo de Transformers
    if tipo_modelo == "transformers":
        for ent in entidades:
            tipo = ent.get('entity', 'O')  # Usa el campo 'entity' o 'O' por defecto
            conteo[tipo] = conteo.get(tipo, 0) + 1  # Acumula el conteo por tipo

    #  Caso: salida estilo Gemini (string con entidades tipo: "Diego → PERSONA")
    elif tipo_modelo == "gemini":
        if isinstance(entidades, str):  # Asegura que sea texto
            lineas = entidades.strip().split('\n')  # Divide en líneas
            for linea in lineas:
                partes = linea.split("→")  # Separa entidad y tipo
                if len(partes) >= 2:
                    tipo = partes[1].strip().upper()  # Obtiene tipo y lo normaliza
                    conteo[tipo] = conteo.get(tipo, 0) + 1  # Acumula

    return conteo  # Devuelve diccionario con el conteo por tipo

# 2. Función para medir tiempo de ejecución de un modelo NER
def medir_tiempo_procesamiento(texto, modelo, tipo_modelo="transformers"):
    """
    Devuelve el tiempo (en segundos) que tarda en procesar un texto con el modelo.
    """
    inicio = time.perf_counter()  # Marca el inicio
    contar_entidades_por_tipo(texto, modelo, tipo_modelo)  # Ejecuta análisis
    fin = time.perf_counter()  # Marca el final

    return round(fin - inicio, 4)  # Devuelve tiempo redondeado

# 3. Comparativa entre modelos: ejecución y cantidad de entidades

# --  Análisis con modelo Transformers (local) --
print(" Modelo: Transformers")

# Medimos tiempo para el primer texto con Transformers
tiempo_transformers_1 = medir_tiempo_procesamiento(texto_ejemplo_1, ner_pipeline, "transformers")
# Medimos tiempo para el segundo texto
tiempo_transformers_2 = medir_tiempo_procesamiento(texto_ejemplo_2, ner_pipeline, "transformers")

# Contamos entidades por tipo en ambos textos
conteo_transformers_1 = contar_entidades_por_tipo(texto_ejemplo_1, ner_pipeline, "transformers")
conteo_transformers_2 = contar_entidades_por_tipo(texto_ejemplo_2, ner_pipeline, "transformers")

# Mostramos resultados por consola
print(f" Tiempo (Texto 1): {tiempo_transformers_1} s — Entidades: {conteo_transformers_1}")
print(f" Tiempo (Texto 2): {tiempo_transformers_2} s — Entidades: {conteo_transformers_2}")

# --  Análisis con Gemini (si está disponible) --
try:
    if cliente_gemini:
        print("\n Modelo: Gemini")

        # Medimos tiempo para ambos textos
        tiempo_gemini_1 = medir_tiempo_procesamiento(texto_ejemplo_1, analizar_entidades_gemini, "gemini")
        tiempo_gemini_2 = medir_tiempo_procesamiento(texto_ejemplo_2, analizar_entidades_gemini, "gemini")

        # Contamos entidades extraídas con Gemini
        conteo_gemini_1 = contar_entidades_por_tipo(texto_ejemplo_1, analizar_entidades_gemini, "gemini")
        conteo_gemini_2 = contar_entidades_por_tipo(texto_ejemplo_2, analizar_entidades_gemini, "gemini")

        # Mostramos resultados por consola
        print(f" Tiempo (Texto 1): {tiempo_gemini_1} s — Entidades: {conteo_gemini_1}")
        print(f" Tiempo (Texto 2): {tiempo_gemini_2} s — Entidades: {conteo_gemini_2}")
    else:
        print("\n API de Gemini no disponible. Saltando análisis.")
except NameError:
    print("\n Variable 'cliente_gemini' no definida. Skipping Gemini.")


 Modelo: Transformers
 Tiempo (Texto 1): 1.1207 s — Entidades: {'I-PER': 3, 'I-LOC': 6, 'I-ORG': 14}
 Tiempo (Texto 2): 1.2035 s — Entidades: {'I-LOC': 17, 'I-PER': 1}

 Modelo: Gemini
 Tiempo (Texto 1): 1.1367 s — Entidades: {'PERSONA': 2, 'LUGAR': 3, 'ORGANIZACIÓN': 2}
 Tiempo (Texto 2): 0.832 s — Entidades: {'LUGAR': 5, 'PERSONA': 1}


*Transformers ofrece más detección y detalle técnico, mientras que Gemini entrega una salida más limpia y directa, aunque menos extensa.*



---


## -- Ejercicio 3: Extensiones Avanzadas (AVANZADO) --


---





In [ ]:
# 1. Implementar procesamiento en lote de múltiples textos
def procesar_lote_de_textos(textos, modelo, tipo_modelo="transformers"):
    """Procesa un lote de textos y devuelve la lista de entidades encontradas por texto"""
    resultados = []
    for texto in textos:
        # For transformers, we need the raw list of entities, not the counts
        if tipo_modelo == "transformers":
            # Apply the model directly, which returns a list of entity dicts
             if not modelo or not texto.strip():
                 resultados.append([]) # Append empty list if no model or text
             else:
                # The ner_pipeline returns a list of dictionaries
                resultados.append(modelo(texto))
        elif tipo_modelo == "gemini":
             # For Gemini, we'd need to parse the string output into a list of entity dicts
             # This requires a change in the analyze_entities_gemini function or adding a parsing step here
             # For now, let's assume we only export transformers results with this function structure
             print("Exportación CSV para Gemini no implementada en esta función.")
             resultados.append([]) # Append empty list

    return resultados

# Lista de textos para procesar
lote_de_textos = [texto_ejemplo_1, texto_ejemplo_2, texto_ejemplo_3, texto_ejemplo_4, texto_ejemplo_5]

# Procesar lote usando Transformers
# Now results_transformers will be a list of lists of entity dictionaries
resultados_transformers_raw = procesar_lote_de_textos(lote_de_textos, ner_pipeline, tipo_modelo="transformers")

# # Example of printing raw results (optional)
# for i, resultado in enumerate(resultados_transformers_raw):
#     print(f"Texto {i+1}: {resultado[:5]}...") # Print first 5 entities

# 2. Crear una función de exportación de resultados a CSV
import csv

def exportar_a_csv(resultados_raw, archivo_nombre):
    """Exporta los resultados de NER (lista de entidades) a un archivo CSV"""
    with open(archivo_nombre, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Update headers based on actual entity dictionary structure
        writer.writerow(['Texto_ID', 'Entidad_Texto', 'Categoria', 'Confianza', 'Inicio', 'Fin'])
        for i, lista_entidades_texto in enumerate(resultados_raw):
            # Iterate through the list of entity dictionaries for each text
            for entidad in lista_entidades_texto:
                # Access the correct keys from the entity dictionary
                entidad_texto = entidad.get('word', '') # Common key for the entity text
                categoria = entidad.get('entity', '') # Common key for the entity type/category
                confianza = entidad.get('score', '') # Common key for confidence score
                inicio = entidad.get('start', '') # Common key for start position
                fin = entidad.get('end', '') # Common key for end position

                writer.writerow([f"Texto {i+1}", entidad_texto, categoria, confianza, inicio, fin])

# Exportar los resultados a un archivo CSV
exportar_a_csv(resultados_transformers_raw, 'resultados_ner.csv')

print(f"Resultados exportados a 'resultados_ner.csv'")


# 3. Desarrollar un sistema de filtrado por tipo de entidad
def filtrar_por_tipo_entidad(resultados_raw, tipo):
    """Filtra las entidades por tipo de una lista de listas de entidades raw"""
    entidades_filtradas = []
    # Iterate through the list of lists of entities
    for lista_entidades_texto in resultados_raw:
        # Iterate through the entity dictionaries in each text's list
        for entidad in lista_entidades_texto:
            # Check the category key
            if entidad.get('entity', '') == tipo: # Use .get for safety
                entidades_filtradas.append(entidad)
    return entidades_filtradas

# Filtrar solo las entidades de tipo 'PER' (personas)
personas_filtradas = filtrar_por_tipo_entidad(resultados_transformers_raw, 'I-PER') # Use 'I-PER' as per the model output structure
print(f"Personas encontradas: {personas_filtradas}")

Resultados exportados a 'resultados_ner.csv'
Personas encontradas: [{'entity': 'I-PER', 'score': np.float32(0.9986106), 'index': 6, 'word': 'Roberto', 'start': 11, 'end': 18}, {'entity': 'I-PER', 'score': np.float32(0.99846065), 'index': 7, 'word': 'González', 'start': 19, 'end': 27}, {'entity': 'I-PER', 'score': np.float32(0.99043006), 'index': 66, 'word': 'Mariana', 'start': 209, 'end': 216}, {'entity': 'I-PER', 'score': np.float32(0.9852653), 'index': 27, 'word': 'Diego', 'start': 78, 'end': 83}, {'entity': 'I-PER', 'score': np.float32(0.9980317), 'index': 3, 'word': 'Ana', 'start': 5, 'end': 8}, {'entity': 'I-PER', 'score': np.float32(0.99945015), 'index': 4, 'word': 'Martínez', 'start': 9, 'end': 17}, {'entity': 'I-PER', 'score': np.float32(0.46797514), 'index': 48, 'word': 'El', 'start': 136, 'end': 138}, {'entity': 'I-PER', 'score': np.float32(0.9956927), 'index': 1, 'word': 'Carlos', 'start': 1, 'end': 7}, {'entity': 'I-PER', 'score': np.float32(0.99504787), 'index': 2, 'word':

*muestran la lista de personas identificadas por el modelo de Transformers a partir de varios textos de ejemplo. Cada entidad reconocida incluye su tipo (I-PER para personas), la palabra detectada, su índice en el texto, la posición de inicio y fin, y la confianza del modelo (score), expresada como probabilidad entre 0 y 1.*

Personas detectadas (I-PER):

+ Nombres correctamente identificados: Roberto González, Mariana, Diego, Ana Martínez, Carlos Fernández.

+ Alta confianza (score > 0.98) en la mayoría.

+ Falsos positivos: "El", "Tam" con score bajo (< 0.7).

+ Total: 11 entidades detectadas como personas.

Observación:

+ El modelo acierta en nombres comunes, pero puede etiquetar mal palabras fuera de contexto.

Exportación:

+ Los resultados se guardaron en resultados_ner.csv para análisis posterior

## -- Proyecto Integrador --

In [ ]:
# Elegí el "Analizador de noticias argentinas" como proyecto integrador.

# The ner_pipeline object is already defined and loaded in a previous cell
# from transformers import pipeline # Already imported
# import torch # Already imported

def analizador_de_noticias(texto, ner_pipeline):
    """Extrae personas y lugares de artículos de noticias argentinas usando el pipeline de transformers"""
    # Get the raw list of entity dictionaries from the pipeline
    if not ner_pipeline or not texto.strip():
        return [], [] # Return empty lists if no pipeline or text

    raw_entidades = ner_pipeline(texto)

    # The filtrar_por_tipo_entidad function expects a list of lists,
    # but ner_pipeline gives a single list of dictionaries for one text.
    # We can wrap the single list in another list to match the expected input structure
    # or modify filtrar_por_tipo_entidad.
    # Let's modify filtrar_por_tipo_entidad to handle a single list or a list of lists.

    # Let's rewrite filtrar_por_tipo_entidad to be more flexible
    def filtrar_entidades_flexible(entidades_list, tipo):
        """Filtra las entidades por tipo from a list of entity dicts."""
        entidades_filtradas = []
        # Iterate directly through the entity dictionaries in the list
        for entidad in entidades_list:
            # Check the category key
            if entidad.get('entity', '') == tipo: # Use .get for safety
                entidades_filtradas.append(entidad)
        return entidades_filtradas

    # Now use the flexible filtering function
    personas = filtrar_entidades_flexible(raw_entidades, 'I-PER') # Use 'I-PER' based on model output
    lugares = filtrar_entidades_flexible(raw_entidades, 'I-LOC') # Use 'I-LOC' based on model output

    return personas, lugares

# Texto de ejemplo para análisis de noticias
texto_noticia = """
El presidente Alberto Fernández anunció nuevas medidas económicas en la Casa Rosada.
Acompañado de ministros como Martín Guzmán, dio detalles sobre el plan para la reactivación económica.
"""

# Pass the ner_pipeline object to the function
personas, lugares = analizador_de_noticias(texto_noticia, ner_pipeline)
print(f"Personas: {personas}")
print(f"Lugares: {lugares}")

Personas: [{'entity': 'I-PER', 'score': np.float32(0.9993493), 'index': 4, 'word': 'Alberto', 'start': 15, 'end': 22}, {'entity': 'I-PER', 'score': np.float32(0.9994324), 'index': 5, 'word': 'Fernández', 'start': 23, 'end': 32}, {'entity': 'I-PER', 'score': np.float32(0.99908376), 'index': 38, 'word': 'Martín', 'start': 115, 'end': 121}, {'entity': 'I-PER', 'score': np.float32(0.99873704), 'index': 39, 'word': 'G', 'start': 122, 'end': 123}, {'entity': 'I-PER', 'score': np.float32(0.955326), 'index': 40, 'word': '##uz', 'start': 123, 'end': 125}, {'entity': 'I-PER', 'score': np.float32(0.99608684), 'index': 41, 'word': '##mán', 'start': 125, 'end': 128}]
Lugares: [{'entity': 'I-LOC', 'score': np.float32(0.9807742), 'index': 23, 'word': 'Casa', 'start': 73, 'end': 77}, {'entity': 'I-LOC', 'score': np.float32(0.97945774), 'index': 24, 'word': 'Rosa', 'start': 78, 'end': 82}, {'entity': 'I-LOC', 'score': np.float32(0.98161834), 'index': 25, 'word': '##da', 'start': 82, 'end': 84}]


**entidades extraídas del texto**

Personas detectadas (I-PER):

Correctas:

+ Alberto Fernández

+ Martín Guzmán (reconocido en sub-palabras: Martín, G, ##uz, ##mán)

+ Total: 6 entidades, aunque 3 corresponden a un solo nombre compuesto mal tokenizado.

+ Confianza: Muy alta (scores entre 0.95 y 0.999).

Lugares detectados (I-LOC):

+ Detectado como tokens separados: Casa, Rosa, ##da

+ En conjunto, identifican correctamente Casa Rosada.

+ Confianza: Alta (aprox. 0.98 cada uno).

Observación:

+ El modelo identifica bien los nombres y lugares clave, pero fragmenta palabras compuestas debido al tokenizador subword (WordPiece).

##-- Preguntas de Reflexión

 1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
 - **Transformers**: Ventajas: precisión, localización, flexibilidad. Desventajas: más lento, requiere más recursos.
 - **Gemini**: Ventajas: rapidez, integración con otras herramientas de Google. Desventajas: menos control sobre el modelo, dependencia de una API externa.

 2. ¿En qué casos usarías un modelo local vs una API?
 - Usaría un modelo local cuando necesite control total sobre el proceso y cuando se trate de proyectos con grandes volúmenes de datos.
 - Usaría una API cuando la rapidez sea más importante o cuando no tenga los recursos para ejecutar un modelo local.

3. ¿Cómo evaluarías la precisión de los resultados?
- Mediría la precisión comparando las entidades identificadas con un conjunto de datos etiquetados manualmente y calculando métricas como precisión, recall y F1.

4. ¿Qué consideraciones éticas debemos tener en cuenta?
 - Considerar la privacidad de los datos, evitar sesgos en los modelos y asegurar el uso ético de las predicciones, especialmente en contextos sensibles.

5. ¿Cómo escalarías esta solución para procesar miles de documentos?
 - Utilizaría un enfoque distribuido con procesamiento paralelo, optimizando los tiempos de carga del modelo y usando almacenamiento en la nube para manejar grandes volúmenes de datos.

